# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [ ]:
!pip install ultralytics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 33.6 MB/s eta 0:00:00


In [ ]:
import cv2
from ultralytics import YOLO, solutions

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [ ]:
model = YOLO ('/content/YOLO_Model.pt')

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [ ]:
cap = cv2.VideoCapture('/content/YOLO_Video.mp4')
assert cap.isOpened(), 'Error reading video file'

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [ ]:
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(cap.get(cv2.CAP_PROP_FPS))
print("Frame Height",height)
print("Frame Width",width)
print("Frames",fps)

Frame Height 1080
Frame Width 1920
Frames 29


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [ ]:
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


In [ ]:
video_writer = cv2.VideoWriter('Exam_output3.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [ ]:
def get_box_coordinates (prediction, cell_size, anchor_boxes):
    x, y, width, height = prediction[:4]
    x_center = x * cell_size
    y_center = y * cell_size
    box_width = width * anchor_boxes[0]
    box_height = height * anchor_boxes[1]


    x_min = int (x_center - box_width / 2)
    y_min = int (y_center - box_height / 2)
    x_max = int (x_center + box_width / 2)
    y_max = int (y_center + box_height / 2)
    return x_min, y_min, x_max, y_max

In [ ]:
cell_size = 32
anchor_boxes = [10, 13]
filtered_predictions = []
bounding_boxes = [get_box_coordinates(box, cell_size, anchor_boxes) for box in filtered_predictions]

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [ ]:
for box in bounding_boxes:
    result = {'class': box[0], 'confidence': box[1], 'bbox': box[2]}
    print(f"Class: {result['class']}, Confidence: {result['confidence']}, Bounding Box: {result['bbox']}")

In [ ]:
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print('Video processing completed or no more frames.')
        break


    results = model(im0)


    left_count = 0
    right_count = 0
    bottom_count = 0
    top_count = 0
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0].item()
            cls_id = int(box.cls[0].item())

            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2 )/2)

            if cx < width // 2:
                left_count += 1
            else:
                right_count += 1

            if cy < height // 2:
                bottom_count += 1
            else:
                top_count += 1

            cv2.rectangle(im0, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f'Conf: {conf:.2f}, Class: {cls_id}'
            cv2.putText(im0, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)


    cv2.line(im0, (w // 2, 0), (w // 2, h), (255, 0, 0), 2)


    cv2.putText(im0, f'Left: {left_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(im0, f'Right: {right_count}', (width - 250, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(im0, f'bottom: {bottom_count}', (height - 250, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(im0, f'top: {top_count}', (height - 250, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    video_writer.write(im0)



    cap.release()
video_writer.release()
cv2.destroyAllWindows()


0: 384x640 4 buss, 88 cars, 1 truck, 3 vans, 2603.0ms
Speed: 4.8ms preprocess, 2603.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)


In [ ]:
 cap.release()
video_writer.release()
cv2.destroyAllWindows()

## Save and Submit Your Work

Submit both the notebook and the output video